In [56]:
import pandas as pd
import statsmodels.api as sm

## 행정구 별 생활인구, 관광거리 수, 확진자 수 데이터 불러오기 및 수정

### 생활인구

In [35]:
# 생활인구
local_p = pd.read_csv('C:/Users/shine/Desktop/BigData/Mentoring/0919/preprocess data/서울 20~30대 생활인구(6~8월).csv')
local_p = local_p.set_index('날짜')
local_p

,LP(강남),LP(강동),LP(강북),LP(강서),LP(관악),LP(광진),LP(구로),LP(금천),LP(노원),LP(도봉),...,LP(성동),LP(성북),LP(송파),LP(양천),LP(영등포),LP(용산),LP(은평),LP(종로),LP(중구),LP(중랑)
날짜,,,,,,,,,,,,,,,,,,,,,
2020-06-01,14292,7504,5988,8729,9399,8701,8227,8394,7083,5045,...,7155,6548,8668,5523,9690,6492,7522,6022,7237,6119
2020-06-02,14566,7488,5987,8696,9408,8675,8223,8415,7075,5023,...,7184,6563,8660,5530,9656,6500,7512,6177,7308,6117
2020-06-03,14649,7508,5964,8667,9366,8676,8171,8402,7047,5014,...,7190,6559,8681,5472,9674,6523,7470,6312,7328,6080
2020-06-04,14683,7455,5940,8650,9356,8675,8166,8364,7051,4995,...,7211,6533,8667,5464,9670,6560,7461,6298,7349,6075
2020-06-05,14487,7454,5924,8601,9322,8748,8111,8183,7009,4990,...,7182,6469,8686,5444,9680,6555,7441,6282,7225,6079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-27,13418,7630,6039,8844,9493,8570,8137,8224,6966,5087,...,7137,6351,8623,5432,9342,6266,7798,5399,6629,6244
2020-08-28,13402,7533,5934,8674,9301,8487,7984,8068,6869,5005,...,7074,6246,8511,5357,9252,6252,7661,5375,6595,6157
2020-08-29,10445,7927,6400,8893,9955,8813,7691,6881,7306,5373,...,6584,6507,8412,5631,8176,5910,8347,4280,4568,6712


### 구별 일일 확진자 수

In [16]:
# 일별, 행정구별 확진자 수
cases_p = pd.read_excel('C:/Users/shine/Desktop/BigData/Mentoring/0919/preprocess data/확진자 전처리.xlsx')
# 일별 내 구별 확진자 파악을 위해 열 추가
cases_p['인원 수'] = 1
# 날짜 내림차순 정렬
cases_p = cases_p.sort_values('확진일자', ascending=True)
# 날짜 형태 수정
cases_p['확진일자'] = cases_p['확진일자'].str.replace('.', '')
cases_p['확진일자'] = '20200' + cases_p['확진일자']
cases_p['확진일자'] = cases_p['확진일자'].astype(int)
cases_p = cases_p.rename(columns={'확진일자':'날짜', '지역':'행정구'})
# 날짜, 행정구 기준으로 일일 확진자 수 합계를 구함
cases_p = cases_p.groupby(['날짜','행정구']).sum()
# 행정구, 날짜 기준으로 오름차순 정렬
cases_p = cases_p.sort_values(by=['행정구','날짜'], ascending=[True,True])
cases_p = cases_p.reset_index()
cases_p['날짜'] = pd.to_datetime(cases_p['날짜'], format='%Y%m%d')
cases_p = cases_p.set_index('날짜')
cases_p

,행정구,인원 수
날짜,,
2020-06-02,강남구,1
2020-06-03,강남구,2
2020-06-16,강남구,1
2020-06-17,강남구,1
2020-06-22,강남구,1
...,...,...
2020-08-05,포천,1
2020-08-06,하남,1
2020-08-08,하남,1


### 구별 관광거리 수

In [31]:
# 관광거리 수(인코딩 문제로 engine 옵션 설정 )
street = pd.read_csv('C:/Users/shine/Desktop/BigData/Mentoring/0919/preprocess data/구별 관광거리 수.csv', engine='python')
street = street.rename(columns={'Unnamed: 0':'행정구','법정 구':'거리 수'})
street.head(3)

,행정구,거리 수
0,종로구,22
1,중구,20
2,양천구,11


## 생활인구 + 일일 행정구별 확진자 수 데이터 프레임 병합

In [67]:
# 구별 확진자수
district = cases_p

# 구별 생활인구 컬럼 만들기
gn = district[district['행정구'].isin(['강남구'])]
gn = pd.DataFrame(data = gn[['인원 수']])
df = local_p.join(gn, how='left')
df = df.rename(columns={'인원 수':'CA(강남)'})

gd = district[district['행정구'].isin(['강동구'])]
gd = pd.DataFrame(data = gd[['인원 수']])
df = df.join(gd, how='left')
df = df.rename(columns={'인원 수':'CA(강동)'})

gb = district[district['행정구'].isin(['강북구'])]
gb = pd.DataFrame(data = gb[['인원 수']])
df = df.join(gb, how='left')
df = df.rename(columns={'인원 수':'CA(강북)'})

gs = district[district['행정구'].isin(['강서구'])]
gs = pd.DataFrame(data = gs[['인원 수']])
df = df.join(gs, how='left')
df = df.rename(columns={'인원 수':'CA(강서)'})

ga = district[district['행정구'].isin(['관악구'])]
ga = pd.DataFrame(data = ga[['인원 수']])
df = df.join(ga, how='left')
df = df.rename(columns={'인원 수':'CA(관악)'})

gj = district[district['행정구'].isin(['광진구'])]
gj = pd.DataFrame(data = gj[['인원 수']])
df = df.join(gj, how='left')
df = df.rename(columns={'인원 수':'CA(광진)'})

gr = district[district['행정구'].isin(['구로구'])]
gr = pd.DataFrame(data = gr[['인원 수']])
df = df.join(gr, how='left')
df = df.rename(columns={'인원 수':'CA(구로)'})

gc = district[district['행정구'].isin(['금천구'])]
gc = pd.DataFrame(data = gc[['인원 수']])
df = df.join(gc, how='left')
df = df.rename(columns={'인원 수':'CA(금천)'})

nw = district[district['행정구'].isin(['노원구'])]
nw = pd.DataFrame(data = nw[['인원 수']])
df = df.join(nw, how='left')
df = df.rename(columns={'인원 수':'CA(노원)'})

db = district[district['행정구'].isin(['도봉구'])]
db = pd.DataFrame(data = db[['인원 수']])
df = df.join(db, how='left')
df = df.rename(columns={'인원 수':'CA(도봉)'})

ddm = district[district['행정구'].isin(['동대문구'])]
ddm = pd.DataFrame(data = ddm[['인원 수']])
df = df.join(ddm, how='left')
df = df.rename(columns={'인원 수':'CA(동대문)'})

dj = district[district['행정구'].isin(['동작구'])]
dj = pd.DataFrame(data = dj[['인원 수']])
df = df.join(dj, how='left')
df = df.rename(columns={'인원 수':'CA(동작)'})

mp = district[district['행정구'].isin(['마포구'])]
mp = pd.DataFrame(data = mp[['인원 수']])
df = df.join(mp, how='left')
df = df.rename(columns={'인원 수':'CA(마포)'})

sdm = district[district['행정구'].isin(['서대문구'])]
sdm = pd.DataFrame(data = sdm[['인원 수']])
df = df.join(sdm, how='left')
df = df.rename(columns={'인원 수':'CA(서대문)'})

sc = district[district['행정구'].isin(['서초구'])]
sc = pd.DataFrame(data = sc[['인원 수']])
df = df.join(sc, how='left')
df = df.rename(columns={'인원 수':'CA(서초)'})

sd = district[district['행정구'].isin(['성동구'])]
sd = pd.DataFrame(data = sd[['인원 수']])
df = df.join(sd, how='left')
df = df.rename(columns={'인원 수':'CA(성동)'})

sb = district[district['행정구'].isin(['성북구'])]
sb = pd.DataFrame(data = sb[['인원 수']])
df = df.join(sb, how='left')
df = df.rename(columns={'인원 수':'CA(성북)'})

sp = district[district['행정구'].isin(['송파구'])]
sp = pd.DataFrame(data = sp[['인원 수']])
df = df.join(sp, how='left')
df = df.rename(columns={'인원 수':'CA(송파)'})

yc = district[district['행정구'].isin(['양천구'])]
yc = pd.DataFrame(data = yc[['인원 수']])
df = df.join(yc, how='left')
df = df.rename(columns={'인원 수':'CA(양천)'})

ydp = district[district['행정구'].isin(['영등포구'])]
ydp = pd.DataFrame(data = ydp[['인원 수']])
df = df.join(ydp, how='left')
df = df.rename(columns={'인원 수':'CA(영등포)'})

ys = district[district['행정구'].isin(['용산구'])]
ys = pd.DataFrame(data = ys[['인원 수']])
df = df.join(ys, how='left')
df = df.rename(columns={'인원 수':'CA(용산)'})

ep = district[district['행정구'].isin(['은평구'])]
ep = pd.DataFrame(data = ep[['인원 수']])
df = df.join(ep, how='left')
df = df.rename(columns={'인원 수':'CA(은평)'})

jr1 = district[district['행정구'].isin(['종로구'])]
jr1 = pd.DataFrame(data = jr1[['인원 수']])
df = df.join(jr1, how='left')
df = df.rename(columns={'인원 수':'CA(종로)'})

j = district[district['행정구'].isin(['중구'])]
j = pd.DataFrame(data = j[['인원 수']])
df = df.join(j, how='left')
df = df.rename(columns={'인원 수':'CA(중구)'})

jr2 = district[district['행정구'].isin(['중랑구'])]
jr2 = pd.DataFrame(data = jr2[['인원 수']])
df = df.join(jr2, how='left')
df = df.rename(columns={'인원 수':'CA(중랑)'})

# 결측값 처리
df = df.fillna(0).astype(int)
df.to_csv('C:/Users/shine/Desktop/BigData/Mentoring/0919/preprocess data/6~8월 서울시 구별 현황(생활인구+일일 확진자).csv', encoding='utf-8-sig')
df

,LP(강남),LP(강동),LP(강북),LP(강서),LP(관악),LP(광진),LP(구로),LP(금천),LP(노원),LP(도봉),...,CA(성동),CA(성북),CA(송파),CA(양천),CA(영등포),CA(용산),CA(은평),CA(종로),CA(중구),CA(중랑)
날짜,,,,,,,,,,,,,,,,,,,,,
2020-06-01,14292,7504,5988,8729,9399,8701,8227,8394,7083,5045,...,0,0,1,3,3,0,0,0,0,0
2020-06-02,14566,7488,5987,8696,9408,8675,8223,8415,7075,5023,...,1,0,0,5,0,0,0,0,0,0
2020-06-03,14649,7508,5964,8667,9366,8676,8171,8402,7047,5014,...,0,0,0,0,1,1,0,0,0,0
2020-06-04,14683,7455,5940,8650,9356,8675,8166,8364,7051,4995,...,0,0,0,4,0,1,0,0,0,0
2020-06-05,14487,7454,5924,8601,9322,8748,8111,8183,7009,4990,...,1,0,1,4,1,2,2,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-27,13418,7630,6039,8844,9493,8570,8137,8224,6966,5087,...,0,7,16,3,7,4,9,3,2,4
2020-08-28,13402,7533,5934,8674,9301,8487,7984,8068,6869,5005,...,0,6,14,3,5,1,1,0,3,4
2020-08-29,10445,7927,6400,8893,9955,8813,7691,6881,7306,5373,...,2,7,12,8,13,0,9,4,2,7


In [77]:
lp_gn = df['LP(강남)']
ca_gn = df['CA(강남)']
df_gn = pd.DataFrame({'LP(강남)':lp_gn, 'CA(강남)':ca_gn})
df_gn
# Summary 및 Modeling
reg = sm.OLS.from_formula('lp_gn ~ ca_gn', df_gn).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lp_gn   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.270
Date:                Mon, 05 Oct 2020   Prob (F-statistic):              0.263
Time:                        21:33:31   Log-Likelihood:                -821.38
No. Observations:                  92   AIC:                             1647.
Df Residuals:                      90   BIC:                             1652.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.354e+04    223.421     60.596      0.000    1.31e+04     1.4e+04
ca_gn       -106.2045     94.248     -1.127      0.263    -293.445      81.036
==============================================================================
Omnibus:                       18.873   Durbin-Watson:                   1.269
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               16.477
Skew:                          -0.943   Prob(JB):                     0.000264
Kurtosis:                       2.137   Cond. No.                         2.90
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### 항목별 설명

* R-squared(R제곱)
: 앞서 회귀분석을 실시한 "lp_gn  = B0 + B1 * ca_gn" 모델이 df_gn 데이터프레임을 얼마나 설명해주는지 모델식의 적합성을 말해주는 것으로, 0~1의 값으로 0은 모델의 설명력이 전혀 없는 상태, 1은 모델이 완벽하게 데이터를 설명해주는 상태임. 현재 결과는 선형 회귀분석 모델이 lp_gn 변동성의 1.4%를 설명한다는 의미임.(사회과학에서는 보통 0.4 이상 이면 괜찮은 모델)

* coef(coefficient, 계수)
: intercept는 1.354e+04, ca_gn는	-106.2045임. 이 계수값을 선형 회귀분석 모델에 대입하면 "lp_gn = 1.354e+04 - 106.2045 * ca_gn"의 식이 산출된 것을 확인할 수 있음.

* P>|t|(유의확률)
: 독립변수의 유의확률임. 보통 독립변수가 95%의 신뢰도를 가져야 유의미하다고 판단하고, 이때 독립변수의 유의확률은 0.05보다 작은 값이 산출됨. 즉, 독립변수의 유의확률이 0.05보다 작으면, 독립변수가 종속변수에 영향을 미치는 것이 유의미하다고 보는데, 위 경우 ca_gn의 유의확률은 0.263으로 ca_gn는 lp_gn에 무의미함.

* Durbin-Watson(더빈왓슨, DW검정)
: DW 검정은 잔차의 독립성을 확인할 수 있는 수치로, 0이면 잔차들이 양의 자기상관을 갖고, 2이면 자기상관이 없는 독립성을 갖고, 4이면 잔차들이 음의 자기상관을 갖는다고 해석함. 보통 1.5 ~ 2.5사이이면 독립으로 판단하고 회귀모형이 적합하다는 것을 의미하는데, DW검정값이 0 또는 4에 가깝다는 것은 잔차들이 자기상관을 가지고 있다는 의미이고, 이는 t값, F값, R제곱을 실제보다 증가시켜 실제로 유의미하지 않은 결과를 유의미한 결과로 왜곡하게 됨.

* No.observations
: 몇 개의 데이터 쌍으로 회귀분석을 실시했는지를 의미하는 것으로, 여기서는 92개의 데이터 쌍을 가지고 회귀분석을 실시했다는 뜻임.

* Df Model
: 회귀분석의 "예측변수의 숫자(k)"를 의미함. 회귀분석의 전체 파라미터는 1개의 종속변수를 포함하므로 Df Model은 다른 식으로는 "회귀분석 전체 파라미터 숫자 - 1" 이다. 예시 회귀모형에서 예측변수는 "ca_gn" 1개이므로 Df Model은 1임.

* Df Residuals
: "No.observations - (Df Model + 1)"로 산출하고, 전체 관찰데이터의 수에서 회귀모형의 전체 파라미터의 수를 뺀 값임. 92 - 2 = 90으로 산출.